<h1><center><font color='red'>Argumentation Project</font></center></h1>

<h2><font color='green'>Guidelines</font></h2>

Here you will find the three contexts: complete, (with and without id) and the subcontexts for and against (Thanks Laurine!).<br>

The original micro texts and arg structures are found here: https://github.com/peldszus/arg-microtexts-multilayer<br>

DESCRIPTION: cf. slides<br>

Questions & texts: see .xml in https://github.com/peldszus/arg-microtexts/tree/master/corpus/en<br>

<br>Arg. structures: see .pdf in the same page

<br>There is an additional file (arg_attr_patterns.json) that contains the correspondence between the structure identifier arg and the corresponding structure:

<br>=> json file {k:v} with k the identifier, and v the structure (a character string) in the following format:

<ul>
    <li>t # arg_id // the first line contains the structure identifier (arg_id)</li>
    <li>v v_id v_label // each line starting with v describes a node v_id and its label v_label (no label on the nodes, I put "_")</li>
    <li>e src_id trg_id e_label // each line starting with e describes an arc between src_id and trg_id (two previously defined nodes) and its label e_label</li>
</ul>

<br>The goal of the project is to classify microtexts w.r.t. the argumentation structures that they contain. The classification should rely on the hypotheses (for, against, falsified generalizations) that you'll mine, and an analysis is expected. Reference to the descriptions given in the page above is required. You should sample a few examples for testing your classifier. Compare with more "classical" pattern mining approaches (seen last year). This is an exploratory project!

<br>You'll have to form groups of a maximum of 3 (you can choose the same groups), and write a  short report with your findings and analysis.

<br>Following the discussion with the student representatives, the new Deadline is 28 of February 2022 (midnight)!

<h2><font color='green'>Initialisation</font></h2>

In [1]:
# Importation cell
import pandas as pd
import typing as t
import json
import os
from lxml import etree


<h2><font color='green'>Dataframe creation and observations</font></h2>

<h3><font color='orange'><center>arg_context_con.csv<center></font></h3>

In [2]:
df_arg_context_con = pd.read_csv('Data/arg_context_con.csv',sep=';')
df_arg_context_con.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_k017_con,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,micro_b027_con,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,micro_k003_con,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_k016_con,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,micro_b018_con,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>arg_context_glob.csv<center></font></h3>

In [3]:
df_arg_context_glob = pd.read_csv('Data/arg_context_glob.csv',sep=';')
df_arg_context_glob.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a89,a90,a91,a92,a93,a94,a95,a96,a97,Unnamed: 98
0,1,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


<h3><font color='orange'><center>arg_context_pro.csv<center></font></h3>

In [4]:
df_arg_context_glob = pd.read_csv('Data/arg_context_pro.csv',sep=';')
df_arg_context_glob.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_b033_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,micro_b026_pro,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,micro_k002_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_b032_pro,1,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
4,micro_b024_pro,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>complete_context_withlabels.csv<center></font></h3>

In [5]:
df_arg_context_glob = pd.read_csv('Data/complete_context_withlabels.csv',sep=';')
col_name = df_arg_context_glob.columns
df_arg_context_glob.head()
l_shape = []
for elt in col_name:
    newDf = df_arg_context_glob.loc[df_arg_context_glob[elt] == 1]
    l_shape.append((elt,newDf.shape[0]))


<h2><font color='green'>Extraction pdf</font></h2>

In [6]:
# we try to get all xml file in only one liste
l_xml = [f'"en/{elt}"' for elt in os.listdir('en') if elt[-3:] == 'xml'] # 112 files
print(l_xml[0])
tree = etree.parse('en/micro_b001.xml')
for elt in tree.getroot().getchildren():
    print(elt.attrib)



"en/micro_b001.xml"
{'id': 'e1'}
{'id': 'e2'}
{'id': 'e3'}
{'id': 'e4'}
{'id': 'e5'}
{'id': 'a1', 'type': 'opp'}
{'id': 'a2', 'type': 'opp'}
{'id': 'a3', 'type': 'pro'}
{'id': 'a4', 'type': 'pro'}
{'id': 'a5', 'type': 'pro'}
{'id': 'c6', 'src': 'e1', 'trg': 'a1', 'type': 'seg'}
{'id': 'c7', 'src': 'e2', 'trg': 'a2', 'type': 'seg'}
{'id': 'c8', 'src': 'e3', 'trg': 'a3', 'type': 'seg'}
{'id': 'c9', 'src': 'e4', 'trg': 'a4', 'type': 'seg'}
{'id': 'c10', 'src': 'e5', 'trg': 'a5', 'type': 'seg'}
{'id': 'c1', 'src': 'a1', 'trg': 'a5', 'type': 'reb'}
{'id': 'c2', 'src': 'a2', 'trg': 'a1', 'type': 'sup'}
{'id': 'c3', 'src': 'a3', 'trg': 'c1', 'type': 'und'}
{'id': 'c4', 'src': 'a4', 'trg': 'c3', 'type': 'add'}


In [7]:
l_xml = [f'"en/{elt}"' for elt in os.listdir('en') if elt[-3:] == 'xml']
tree = etree.parse('en/micro_b006.xml')
for elt in tree.getroot().getchildren():
    print(elt.attrib)

{'id': 'e1'}
{'id': 'e2'}
{'id': 'e3'}
{'id': 'e4'}
{'id': 'e5'}
{'id': 'a1', 'type': 'pro'}
{'id': 'a2', 'type': 'pro'}
{'id': 'a3', 'type': 'pro'}
{'id': 'a4', 'type': 'opp'}
{'id': 'a5', 'type': 'pro'}
{'id': 'c6', 'src': 'e1', 'trg': 'a1', 'type': 'seg'}
{'id': 'c7', 'src': 'e2', 'trg': 'a2', 'type': 'seg'}
{'id': 'c8', 'src': 'e3', 'trg': 'a3', 'type': 'seg'}
{'id': 'c9', 'src': 'e4', 'trg': 'a4', 'type': 'seg'}
{'id': 'c10', 'src': 'e5', 'trg': 'a5', 'type': 'seg'}
{'id': 'c2', 'src': 'a2', 'trg': 'a1', 'type': 'sup'}
{'id': 'c3', 'src': 'a3', 'trg': 'a1', 'type': 'sup'}
{'id': 'c4', 'src': 'a4', 'trg': 'a1', 'type': 'reb'}
{'id': 'c5', 'src': 'a5', 'trg': 'c4', 'type': 'und'}


In [8]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 
newDf = df_arg_context_glob.drop(columns=['Unnamed: 0'])

frq_items = apriori(newDf, min_support = 0.5, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

print(len(rules))
print(rules)

12
   antecedents consequents  antecedent support  consequent support   support  \
8   (a56, a13)       (a34)            0.606742            0.606742  0.606742   
9        (a34)  (a56, a13)            0.606742            0.606742  0.606742   
0        (a34)       (a13)            0.606742            0.685393  0.606742   
7   (a34, a56)       (a13)            0.606742            0.685393  0.606742   
2        (a34)       (a56)            0.606742            0.921348  0.606742   
4        (a57)       (a56)            0.550562            0.921348  0.550562   
6   (a34, a13)       (a56)            0.606742            0.921348  0.606742   
1        (a13)       (a34)            0.685393            0.606742  0.606742   
10       (a13)  (a34, a56)            0.685393            0.606742  0.606742   
3        (a56)       (a34)            0.921348            0.606742  0.606742   
11       (a56)  (a34, a13)            0.921348            0.606742  0.606742   
5        (a56)       (a57)           

In [9]:
rules = rules[['antecedents','consequents']]

rules_list = rules.values.tolist()
rules_list = [[list(elt[0]),list(elt[1])] for elt in rules_list]
rules_list

[[['a56', 'a13'], ['a34']],
 [['a34'], ['a56', 'a13']],
 [['a34'], ['a13']],
 [['a34', 'a56'], ['a13']],
 [['a34'], ['a56']],
 [['a57'], ['a56']],
 [['a34', 'a13'], ['a56']],
 [['a13'], ['a34']],
 [['a13'], ['a34', 'a56']],
 [['a56'], ['a34']],
 [['a56'], ['a34', 'a13']],
 [['a56'], ['a57']]]

In [53]:
rules_list_annot = []
for elt in rules_list:
    rules_list_annot.append([('Antecedent',elt[0]),('Consequents',elt[1])])
print(rules_list_annot)
print(len(rules_list_annot))

[[('Antecedent', ['a56', 'a13']), ('Consequents', ['a34'])], [('Antecedent', ['a34']), ('Consequents', ['a56', 'a13'])], [('Antecedent', ['a34']), ('Consequents', ['a13'])], [('Antecedent', ['a34', 'a56']), ('Consequents', ['a13'])], [('Antecedent', ['a34']), ('Consequents', ['a56'])], [('Antecedent', ['a57']), ('Consequents', ['a56'])], [('Antecedent', ['a34', 'a13']), ('Consequents', ['a56'])], [('Antecedent', ['a13']), ('Consequents', ['a34'])], [('Antecedent', ['a13']), ('Consequents', ['a34', 'a56'])], [('Antecedent', ['a56']), ('Consequents', ['a34'])], [('Antecedent', ['a56']), ('Consequents', ['a34', 'a13'])], [('Antecedent', ['a56']), ('Consequents', ['a57'])]]
12


In [83]:
regle1 = rules_list_annot[0]
liste = []
for elt in regle1:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a56,a13,a34
1,micro_b033_pro,1,1,1
3,micro_k003_con,1,1,1
5,micro_k002_pro,1,1,1
6,micro_k016_con,1,1,1
8,micro_b018_con,1,1,1
12,micro_b031_con,1,1,1
13,micro_k015_unclear,1,1,1
14,micro_k001_pro,1,1,1
16,micro_k029_con,1,1,1
18,micro_b021_con,1,1,1


In [84]:
regle2 = rules_list_annot[1]
liste = []
for elt in regle2:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a34,a56,a13
1,micro_b033_pro,1,1,1
3,micro_k003_con,1,1,1
5,micro_k002_pro,1,1,1
6,micro_k016_con,1,1,1
8,micro_b018_con,1,1,1
12,micro_b031_con,1,1,1
13,micro_k015_unclear,1,1,1
14,micro_k001_pro,1,1,1
16,micro_k029_con,1,1,1
18,micro_b021_con,1,1,1


In [85]:
regle3 = rules_list_annot[2]
liste = []
for elt in regle3:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a34,a13
1,micro_b033_pro,1,1
3,micro_k003_con,1,1
5,micro_k002_pro,1,1
6,micro_k016_con,1,1
8,micro_b018_con,1,1
12,micro_b031_con,1,1
13,micro_k015_unclear,1,1
14,micro_k001_pro,1,1
16,micro_k029_con,1,1
18,micro_b021_con,1,1


In [86]:
regle4 = rules_list_annot[3]
liste = []
for elt in regle4:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a34,a56,a13
1,micro_b033_pro,1,1,1
3,micro_k003_con,1,1,1
5,micro_k002_pro,1,1,1
6,micro_k016_con,1,1,1
8,micro_b018_con,1,1,1
12,micro_b031_con,1,1,1
13,micro_k015_unclear,1,1,1
14,micro_k001_pro,1,1,1
16,micro_k029_con,1,1,1
18,micro_b021_con,1,1,1


In [87]:
regle5 = rules_list_annot[4]
liste = []
for elt in regle5:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a34,a56
1,micro_b033_pro,1,1
3,micro_k003_con,1,1
5,micro_k002_pro,1,1
6,micro_k016_con,1,1
8,micro_b018_con,1,1
12,micro_b031_con,1,1
13,micro_k015_unclear,1,1
14,micro_k001_pro,1,1
16,micro_k029_con,1,1
18,micro_b021_con,1,1


In [88]:
regle6 = rules_list_annot[5]
liste = []
for elt in regle6:
    for sub in elt[1]:
        liste.append(sub)

liste.insert(0,'Unnamed: 0')
df = df_arg_context_glob[liste]
df = df.sort_values(by = liste[1:],ascending = False)
df.head(60)

,Unnamed: 0,a57,a56
1,micro_b033_pro,1,1
5,micro_k002_pro,1,1
9,micro_b024_pro,1,1
13,micro_k015_unclear,1,1
16,micro_k029_con,1,1
18,micro_b021_con,1,1
19,micro_k011_pro,1,1
20,micro_b035_pro,1,1
21,micro_b009_pro,1,1
22,micro_b008_con,1,1
